In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_excel("/kaggle/input/cars-final-version/FINAL_DATASET.xlsx", engine='openpyxl')
data.head()


,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix,Ann_Obtention
0,Renault,Capture,121000,Essence,Manuelle,5.0,Autres,2019,43000DT,NaN
1,Seat,Ibiza,97000,Essence,Manuelle,4.0,Compacte,2018,43500DT,NaN
2,Mercedes-Benz,Classe A,135000,Diesel,Automatique,6.0,Compacte,2019,93000DT,NaN
3,Peugeot,Expert,154000,Diesel,Manuelle,5.0,Utilitaire,2019,57000DT,NaN
4,Mercedes-Benz,207D,0,Diesel,Manuelle,0.0,Autres,1981,10000DT,NaN


In [3]:
display(data)

,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix,Ann_Obtention
0,Renault,Capture,121000,Essence,Manuelle,5.0,Autres,2019,43000DT,NaN
1,Seat,Ibiza,97000,Essence,Manuelle,4.0,Compacte,2018,43500DT,NaN
2,Mercedes-Benz,Classe A,135000,Diesel,Automatique,6.0,Compacte,2019,93000DT,NaN
3,Peugeot,Expert,154000,Diesel,Manuelle,5.0,Utilitaire,2019,57000DT,NaN
4,Mercedes-Benz,207D,0,Diesel,Manuelle,0.0,Autres,1981,10000DT,NaN
...,...,...,...,...,...,...,...,...,...,...
14343,Renault,Kadjar,120 000km,Essence,Manuelle,7cv,Citadine,NaN,65 000 DT,11.2017
14344,Toyota,RAV 4,150 000km,Essence,Automatique,9cv,SUV/4x4,NaN,125 000 DT,8.2020
14345,Volkswagen,Polo Sedan,58 000km,Essence,Manuelle,5cv,Berline,NaN,42 000 DT,4.2016
14346,Peugeot,Expert,190 000km,Diesel,Manuelle,9cv,Utilitaire,NaN,Sous leasing 64 800 DT,3.2022


In [4]:
data.shape

(14348, 10)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14348 entries, 0 to 14347
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Marque             14295 non-null  object 
 1   Modèle             14250 non-null  object 
 2   Kilométrage        14315 non-null  object 
 3   Carburant          14283 non-null  object 
 4   Boite Vitesse      14298 non-null  object 
 5   Puissance Fiscale  14202 non-null  object 
 6   Carrosserie        14180 non-null  object 
 7   Année              9206 non-null   object 
 8   Prix               14348 non-null  object 
 9   Ann_Obtention      5057 non-null   float64
dtypes: float64(1), object(9)
memory usage: 1.1+ MB


# **Creating backup copy**

In [6]:
backup=data.copy()

# **Quality**


* price column has unnecessary spaces in some insatances and always a "DT" at the end. 
* some values in the price columns have "sous-leasing" or "non dedouané" along with the price .
* Kilometrage values have unnecessary spaces and sometimes "km" at the end as well as empty values (tnajem houni thot l mean fi blaset l 'nan' ya hamma )
* puissance fiscale : -too many falsified values 
                      -sometimes have 'cv' at the end 
*                       

# **cleaning data**

# **'Prix' column**

In [7]:
display(data['Prix'].sample(n=100))

7886    37 000 DT
988      103000DT
7318    54 500 DT
9558    78 000 DT
4731      21500DT
          ...    
2056      38000DT
4935      10700DT
28        65000DT
6907    25 300 DT
1441      14000DT
Name: Prix, Length: 100, dtype: object

In [8]:
unique_p=data['Prix'].unique()
unique_p

array(['43000DT', '43500DT', '93000DT', ...,
       'Sous leasing                    223 600 DT',
       'Sous leasing                    212 700 DT',
       'Sous leasing                    64 800 DT'], dtype=object)

In [9]:
# # Convert the array to a DataFrame with a single column
# unique_p_df = pd.DataFrame(unique_p, columns=['Unique Values'])

# # Set the maximum number of rows to display
# pd.set_option('display.max_rows', None)

# # Print the DataFrame
# print(unique_p_df)



# defining a function leaving only numerical values 

In [10]:
import re

def extract_numerical_value(text):
    # Use regular expression to find numerical values in the string
    matches = re.findall(r'\d+\s*\d*', text)
    if matches:
        # Replace any whitespace characters with an empty string before joining
        cleaned_matches = [match.replace(' ', '') for match in matches]
        # Join all cleaned matches
        return ''.join(cleaned_matches)
    else:
        return None

In [11]:
data['Prix'] = data['Prix'].apply(lambda x: extract_numerical_value(x))

# delete spacings
data['Prix'] = data['Prix'].str.replace(' ', '')

# Replace 'DT' by '' 
data['Prix'] = data['Prix'].str.replace('DT', '')

# Convert it into numerical 
data['Prix'] = pd.to_numeric(data['Prix'], errors='coerce')

# drop non-numerical values
data.dropna(subset=['Prix'], inplace=True)

# Convert into int 
data['Prix'] = data['Prix'].astype(int)

In [12]:
display(data['Prix'].sample(n=100))

13960     22500
8121      57000
3075      64500
3759      36000
11761     21000
          ...  
2144      42000
7379     112000
10893     50000
1626     180000
12835    103000
Name: Prix, Length: 100, dtype: int64

In [13]:
data['Prix'].unique()


array([ 43000,  43500,  93000,  57000,  10000,  29000,  49500,  39500,
        36800,  24000,  27000,  84000,  74000,  14500, 140000,  18800,
        19500, 285000,   9000,  11500,  15000,  23500,   9800,  12500,
        35000,  42500,  47000,  65000,  31000,  54000,  37800,  32000,
        44800,  30900,  28000,  91000,  15500,  41000,  42000,  49000,
       200000,  98000,  38800, 190000,  50900,  39800,  27500,  85000,
        23000,  33000,  31500, 101200,  41499,  18500,  38000,  77000,
        59000,  79500,  33500,  34000,  46500,   8500,  41500,  46000,
        34800,  67000, 125000,  22000,  52000,  16500,  70000,  36500,
        37000,  40000,  34700,  19000,  22500,  48000,  52500,  38500,
        44500,  16000,  34500,  50000,  82000,  36000,  29500,  78000,
       123456,  95000,  55000,  47498,  45000,  60000,  30500,   6500,
        20000,  62000,  66000,  71000, 215000,  92000,  13000,  26500,
        26900,  21900,  32799,  30000,  21000, 173000,  17500,  10500,
      

In [14]:
data['Prix'].describe()


count     14348.000000
mean      64134.346738
std       63545.168136
min        4700.000000
25%       29000.000000
50%       45000.000000
75%       78000.000000
max      999999.000000
Name: Prix, dtype: float64

In [15]:
data['Prix'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 14348 entries, 0 to 14347
Series name: Prix
Non-Null Count  Dtype
--------------  -----
14348 non-null  int64
dtypes: int64(1)
memory usage: 112.2 KB


this part dropped 0 datapoints from the dataset 

# "kilometrage" column

In [16]:
display(data['Kilométrage'].sample(n=100))

4066      http:N/A
12548          150
4723        208000
8990      90 000km
13919     35 000km
           ...    
4122             0
3302        165000
7462     169 000km
9367     140 000km
6547     148 000km
Name: Kilométrage, Length: 100, dtype: object

In [17]:
print(data['Kilométrage'].unique())

['121000' '97000' '135000' ... '24 500km' '440 000km' '162 500km']


In [18]:
# get rid of spaceing
data['Kilométrage'] = data['Kilométrage'].str.replace(' ', '')

# Replace 'DT' by ''
data['Kilométrage'] = data['Kilométrage'].str.replace('km', '')

# Convert the columns into numerical 
data['Kilométrage'] = pd.to_numeric(data['Kilométrage'], errors='coerce')

# drop the null values 
data.dropna(subset=['Kilométrage'], inplace=True)

# convert all values into "int"
data['Kilométrage'] = data['Kilométrage'].astype(int)


In [19]:
# Drop rows with 'Kilometrage' values larger than 10,000,000 and less than 1,000
data = data[(data['Kilométrage'] <= 10000000) & (data['Kilométrage'] >= 1000)]

In [20]:
data['Kilométrage'].unique()

array([ 121000,   97000,  135000,  154000,    1000,  130000,  180000,
         41500,  226000,   80000,  120000,   74000,  348000,   82000,
         76000,  500000,  137000,   73500,   67000,  203000,   48000,
         40000,   80500,  100000,   56250,  240000,   53000,  125000,
        210000,  110000,   90000,  236000,  122366,   70500,   87000,
        143000,  220000,  126000,   83000,  105000,  123321,   57500,
         62000,   84000,   39000,   18750,  405000,   78500,  157000,
        150000,  187000,   63000,  145000,   21000,  103000,  250000,
        623000,  217000,  234000,   21400,  212000,  141000,  148000,
        320000,  369000,  215000,  230000,   52000,   51000,  197000,
         43000,  155000,   72000,   98000,   73290,  183000,   74086,
        142000,  300000,   99500,   99000,  144000,   64000,  117000,
        147000,   49000,  169000,   47000,   81000,  200500,  181800,
        206000,  280000,  228000,  445000,  270000,  176000,  170000,
         60000,  101

In [21]:
display(data['Kilométrage'].sample(n=100))

8707     128000
13379     37000
8472      69000
1293     153000
3710     130000
          ...  
369      195000
7576     231000
4384      63000
14059    128000
7755     360000
Name: Kilométrage, Length: 100, dtype: int64

In [22]:
data

,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix,Ann_Obtention
0,Renault,Capture,121000,Essence,Manuelle,5.0,Autres,2019,43000,NaN
1,Seat,Ibiza,97000,Essence,Manuelle,4.0,Compacte,2018,43500,NaN
2,Mercedes-Benz,Classe A,135000,Diesel,Automatique,6.0,Compacte,2019,93000,NaN
3,Peugeot,Expert,154000,Diesel,Manuelle,5.0,Utilitaire,2019,57000,NaN
5,Citroen,C4,1000,Essence,Manuelle,6.0,Berline,2013,29000,NaN
...,...,...,...,...,...,...,...,...,...,...
14343,Renault,Kadjar,120000,Essence,Manuelle,7cv,Citadine,NaN,65000,11.2017
14344,Toyota,RAV 4,150000,Essence,Automatique,9cv,SUV/4x4,NaN,125000,8.2020
14345,Volkswagen,Polo Sedan,58000,Essence,Manuelle,5cv,Berline,NaN,42000,4.2016
14346,Peugeot,Expert,190000,Diesel,Manuelle,9cv,Utilitaire,NaN,64800,3.2022


this phase dropped 4336 rows 

# "Puissance Fiscale" column

In [23]:
data['Puissance Fiscale'].unique()

array(['5.0', '4.0', '6.0', '7.0', '9.0', '36.0', '10.0', '0.4', '11.0',
       '8.0', '0.0', '84.0', '12.0', '15.0', '128.0', '18.0', '3.0',
       '16.0', nan, '0.5', '31.0', '17.0', '13.0', '180000.0', '24.0',
       '20.0', '130.0', '103.0', '1.6', '21.0', '1.3', '270.0', '65.0',
       '80.0', '28.0', '19.0', '67.0', '27.0', '1000.0', '0.7', '85.0',
       '1399.0', '1.0', '14.0', '222.0', '44.0', '156.0', '198.0', '69.0',
       '1.9', '23.0', '21000.0', '8000.0', '111508.0', '22.0', '101.0',
       '150.0', '51818.0', '16.115', '1200.0', '70.0', '4.2', '25.0',
       '55.0', '7500.0', '0.6', '-10.0', '43.0', '142.0', '113.0',
       '1198.0', '241.0', '30.0', '66.0', '26.0', '68.0', '280000.0',
       '144.0', '81.0', '2017.0', '2019.0', '3399.0', '116.0', '11500.0',
       '2004.0', '876.0', '2.0', '2021.0', '194.0', '88500.0', '123.0',
       '5cv', '20cv', '9cv', '12cv', '7cv', '10cv', '8cv', '15cv', '6cv',
       '29cv', '17cv', '33cv', '24cv', '4cv', '11cv', '21cv', '18cv',

In [24]:
# Supprimer "cv" de toutes les valeurs de la colonne 'Puissance Fiscale'
data['Puissance Fiscale'] = data['Puissance Fiscale'].str.replace('cv', '')

# Convert
data['Puissance Fiscale'] = pd.to_numeric(data['Puissance Fiscale'], errors='coerce')

# Eliminer les valeurs inférieures à 4
data = data[data['Puissance Fiscale'] > 4]
# Eliminer les valeurs supérieures à  100
data = data[data['Puissance Fiscale'] <= 100]

# Convertir les valeurs en type int
data['Puissance Fiscale'] = data['Puissance Fiscale'].astype(int)

In [25]:
data['Puissance Fiscale'].unique()

array([ 5,  6,  7,  9, 36, 10, 11,  8, 84, 12, 15, 18, 16, 31, 17, 13, 24,
       20, 21, 65, 80, 28, 19, 67, 27, 85, 14, 44, 69, 23, 22, 70,  4, 25,
       55, 43, 30, 66, 26, 68, 81, 29, 33, 32, 47, 34, 42, 41, 48])

In [26]:
data

,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix,Ann_Obtention
0,Renault,Capture,121000,Essence,Manuelle,5,Autres,2019,43000,NaN
2,Mercedes-Benz,Classe A,135000,Diesel,Automatique,6,Compacte,2019,93000,NaN
3,Peugeot,Expert,154000,Diesel,Manuelle,5,Utilitaire,2019,57000,NaN
5,Citroen,C4,1000,Essence,Manuelle,6,Berline,2013,29000,NaN
6,Volkswagen,Polo,130000,Essence,Manuelle,5,Berline,2021,49500,NaN
...,...,...,...,...,...,...,...,...,...,...
14343,Renault,Kadjar,120000,Essence,Manuelle,7,Citadine,NaN,65000,11.2017
14344,Toyota,RAV 4,150000,Essence,Automatique,9,SUV/4x4,NaN,125000,8.2020
14345,Volkswagen,Polo Sedan,58000,Essence,Manuelle,5,Berline,NaN,42000,4.2016
14346,Peugeot,Expert,190000,Diesel,Manuelle,9,Utilitaire,NaN,64800,3.2022


this phase dropped 1126 rows

# "Ann_Obtention" and "Année" 

In [27]:
display(data)

,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix,Ann_Obtention
0,Renault,Capture,121000,Essence,Manuelle,5,Autres,2019,43000,NaN
2,Mercedes-Benz,Classe A,135000,Diesel,Automatique,6,Compacte,2019,93000,NaN
3,Peugeot,Expert,154000,Diesel,Manuelle,5,Utilitaire,2019,57000,NaN
5,Citroen,C4,1000,Essence,Manuelle,6,Berline,2013,29000,NaN
6,Volkswagen,Polo,130000,Essence,Manuelle,5,Berline,2021,49500,NaN
...,...,...,...,...,...,...,...,...,...,...
14343,Renault,Kadjar,120000,Essence,Manuelle,7,Citadine,NaN,65000,11.2017
14344,Toyota,RAV 4,150000,Essence,Automatique,9,SUV/4x4,NaN,125000,8.2020
14345,Volkswagen,Polo Sedan,58000,Essence,Manuelle,5,Berline,NaN,42000,4.2016
14346,Peugeot,Expert,190000,Diesel,Manuelle,9,Utilitaire,NaN,64800,3.2022


In [28]:
data['Ann_Obtention'].unique()

array([    nan,  6.2018, 10.2018,  5.2019,  5.201 ,  7.2012,  1.2013,
        1.202 ,  3.2019,  3.2016, 12.2022,  6.2023,  8.2023,  9.2019,
        3.202 , 12.2023, 11.2022,  1.2021, 10.2019,  1.2009,  5.2014,
        8.2015, 11.2019,  6.2019,  6.2022,  8.2012,  4.201 , 10.2021,
        3.2018,  2.2017, 12.2007,  4.2022,  3.2022,  3.2013, 10.2022,
        5.2017,  7.2014,  1.2015,  4.2023, 10.2012, 11.2013,  1.2016,
       12.2016,  5.2018,  5.2022, 12.202 ,  2.2022,  1.2019, 11.2012,
        6.2021,  1.2018,  6.202 ,  9.2023,  8.202 ,  2.2016,  2.202 ,
        2.2019,  2.2018, 11.2021,  6.2009, 10.2017, 12.2009,  8.2017,
        9.2013, 11.202 ,  4.2014,  8.2022,  9.2014,  7.2015,  9.2012,
        8.2008,  8.2013,  3.2021,  8.2018,  1.201 ,  6.2016, 12.2018,
        9.2021,  2.2012, 12.2019,  6.2015,  5.202 ,  1.2022,  8.2019,
        9.202 ,  2.2013,  1.2012, 10.2013,  5.2013,  7.2013,  9.2018,
        4.2021,  2.2021,  8.2007,  8.201 ,  6.2012, 11.2018,  7.201 ,
        5.2003,  4.2

In [29]:
data["Année"].unique()

array(['2019', '2013', '2021', '2016', '2010', '2018', '2017', '2015',
       '2000', '2014', '2020', '2022', '2009', '2011', '2004', '1993',
       nan, '2006', '2023', '2007', '20', '2008', '2005', '2012', '2024',
       '2001', '2003', '1998', '1997', '1994', '1989', '1995', '1988',
       '1999', '1990', '1996', '45', '1984', '3.2014', '1992', '1985',
       '2002', '1991', '87', '1987', '1982', '20225', '98', '20197',
       '20028', '5', '1', '07.2019', '8.2016', '20205', '02.2019',
       '2011.0', '1953', '12.2018', '11.2019', '21012', '2015.0', '20114',
       '1961', '2018.0', '2017.0', '05.2011', '13', '092021', '2030',
       '2007.0', '135000', '95', '2014.0', '00000', '03.2019', '20175',
       '2006.0', '02.2021'], dtype=object)

In [30]:
# Loop through each row and apply transformations
for index, row in data.iterrows():
    if pd.isnull(row['Année']) and not pd.isnull(row['Ann_Obtention']):
        # Case 2: Extract year from 'mise en circulation' and assign to 'Année'
        year = str(row['Ann_Obtention'])
        data.at[index, 'Année'] = year
    elif pd.isnull(row['Année']) and pd.isnull(row['Ann_Obtention']):
        # Case 3: Drop the row if both 'Année' and 'mise en circulation' are empty
        data.drop(index, inplace=True)

# Drop the column 'mise en circulation'
data.drop(columns=['Ann_Obtention'], inplace=True)


In [31]:
display(data)

,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix
0,Renault,Capture,121000,Essence,Manuelle,5,Autres,2019,43000
2,Mercedes-Benz,Classe A,135000,Diesel,Automatique,6,Compacte,2019,93000
3,Peugeot,Expert,154000,Diesel,Manuelle,5,Utilitaire,2019,57000
5,Citroen,C4,1000,Essence,Manuelle,6,Berline,2013,29000
6,Volkswagen,Polo,130000,Essence,Manuelle,5,Berline,2021,49500
...,...,...,...,...,...,...,...,...,...
14343,Renault,Kadjar,120000,Essence,Manuelle,7,Citadine,11.2017,65000
14344,Toyota,RAV 4,150000,Essence,Automatique,9,SUV/4x4,8.202,125000
14345,Volkswagen,Polo Sedan,58000,Essence,Manuelle,5,Berline,4.2016,42000
14346,Peugeot,Expert,190000,Diesel,Manuelle,9,Utilitaire,3.2022,64800


this dropped 8 rows  

In [32]:
data["Année"].unique()

array(['2019', '2013', '2021', '2016', '2010', '2018', '2017', '2015',
       '2000', '2014', '2020', '2022', '2009', '2011', '2004', '1993',
       '2006', '2023', '2007', '20', '2008', '2005', '2012', '2024',
       '2001', '2003', '1998', '1997', '1994', '1989', '1995', '1988',
       '1999', '1990', '1996', '45', '1984', '3.2014', '1992', '1985',
       '2002', '1991', '87', '1987', '1982', '20225', '98', '20197',
       '20028', '5', '1', '07.2019', '8.2016', '20205', '02.2019',
       '2011.0', '1953', '12.2018', '11.2019', '21012', '2015.0', '20114',
       '1961', '2018.0', '2017.0', '05.2011', '13', '092021', '2030',
       '2007.0', '135000', '95', '2014.0', '00000', '03.2019', '20175',
       '2006.0', '02.2021', '6.2018', '10.2018', '5.2019', '5.201',
       '7.2012', '1.2013', '1.202', '3.2019', '3.2016', '12.2022',
       '6.2023', '8.2023', '9.2019', '3.202', '12.2023', '11.2022',
       '1.2021', '10.2019', '1.2009', '5.2014', '8.2015', '6.2019',
       '6.2022', '8.201

In [33]:
# Define a lambda function to split the string and select the second part
split_second_part = lambda x: x.split('.')[1] if '.' in x else x

# Apply the lambda function to the 'Année' column
data['Année'] = data['Année'].apply(split_second_part)


In [34]:
data

,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix
0,Renault,Capture,121000,Essence,Manuelle,5,Autres,2019,43000
2,Mercedes-Benz,Classe A,135000,Diesel,Automatique,6,Compacte,2019,93000
3,Peugeot,Expert,154000,Diesel,Manuelle,5,Utilitaire,2019,57000
5,Citroen,C4,1000,Essence,Manuelle,6,Berline,2013,29000
6,Volkswagen,Polo,130000,Essence,Manuelle,5,Berline,2021,49500
...,...,...,...,...,...,...,...,...,...
14343,Renault,Kadjar,120000,Essence,Manuelle,7,Citadine,2017,65000
14344,Toyota,RAV 4,150000,Essence,Automatique,9,SUV/4x4,202,125000
14345,Volkswagen,Polo Sedan,58000,Essence,Manuelle,5,Berline,2016,42000
14346,Peugeot,Expert,190000,Diesel,Manuelle,9,Utilitaire,2022,64800


In [35]:
data['Année'].unique()

array(['2019', '2013', '2021', '2016', '2010', '2018', '2017', '2015',
       '2000', '2014', '2020', '2022', '2009', '2011', '2004', '1993',
       '2006', '2023', '2007', '20', '2008', '2005', '2012', '2024',
       '2001', '2003', '1998', '1997', '1994', '1989', '1995', '1988',
       '1999', '1990', '1996', '45', '1984', '1992', '1985', '2002',
       '1991', '87', '1987', '1982', '20225', '98', '20197', '20028', '5',
       '1', '20205', '0', '1953', '21012', '20114', '1961', '13',
       '092021', '2030', '135000', '95', '00000', '20175', '201', '202',
       '2', '197', '1972', '1963'], dtype=object)

In [36]:
# Replace empty strings with NaN
data.replace({'Année': ''}, np.nan, inplace=True)

# Drop rows where 'Année' is empty (NaN)
data.dropna(inplace=True)

In [37]:
# Convert the "Année" column to integers
data['Année'] = data['Année'].astype(int)

# Filter the years between 1000 and 2025
data = data[(data['Année'] >= 1000) & (data['Année'] <= 2025)]

In [38]:
data["Année"].unique()

array([2019, 2013, 2021, 2016, 2010, 2018, 2017, 2015, 2000, 2014, 2020,
       2022, 2009, 2011, 2004, 1993, 2006, 2023, 2007, 2008, 2005, 2012,
       2024, 2001, 2003, 1998, 1997, 1994, 1989, 1995, 1988, 1999, 1990,
       1996, 1984, 1992, 1985, 2002, 1991, 1987, 1982, 1953, 1961, 1972,
       1963])

In [39]:
data

,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix
0,Renault,Capture,121000,Essence,Manuelle,5,Autres,2019,43000
2,Mercedes-Benz,Classe A,135000,Diesel,Automatique,6,Compacte,2019,93000
3,Peugeot,Expert,154000,Diesel,Manuelle,5,Utilitaire,2019,57000
5,Citroen,C4,1000,Essence,Manuelle,6,Berline,2013,29000
6,Volkswagen,Polo,130000,Essence,Manuelle,5,Berline,2021,49500
...,...,...,...,...,...,...,...,...,...
14342,Seat,Leon,162500,Essence,Manuelle,5,Compacte,2014,42000
14343,Renault,Kadjar,120000,Essence,Manuelle,7,Citadine,2017,65000
14345,Volkswagen,Polo Sedan,58000,Essence,Manuelle,5,Berline,2016,42000
14346,Peugeot,Expert,190000,Diesel,Manuelle,9,Utilitaire,2022,64800


this phase dropped 744 rows

# **clean data**

In [40]:
data['Kilométrage'].describe()

count    8.134000e+03
mean     1.455656e+05
std      2.047510e+05
min      1.000000e+03
25%      7.300000e+04
50%      1.230000e+05
75%      1.850000e+05
max      9.999999e+06
Name: Kilométrage, dtype: float64

In [41]:
data.shape

(8134, 9)

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8134 entries, 0 to 14347
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Marque             8134 non-null   object
 1   Modèle             8134 non-null   object
 2   Kilométrage        8134 non-null   int64 
 3   Carburant          8134 non-null   object
 4   Boite Vitesse      8134 non-null   object
 5   Puissance Fiscale  8134 non-null   int64 
 6   Carrosserie        8134 non-null   object
 7   Année              8134 non-null   int64 
 8   Prix               8134 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 635.5+ KB


In [43]:
data.describe(include='all')

,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix
count,8134,8134,8.134000e+03,8134,8134,8134.000000,8134,8134.000000,8134.000000
unique,71,543,NaN,8,2,NaN,11,NaN,NaN
top,Mercedes-Benz,Classe C,NaN,Essence,Manuelle,NaN,Berline,NaN,NaN
freq,956,305,NaN,5938,4465,NaN,2529,NaN,NaN
mean,NaN,NaN,1.455656e+05,NaN,NaN,7.973691,NaN,2015.245881,74092.036513
std,NaN,NaN,2.047510e+05,NaN,NaN,5.139973,NaN,6.072316,63518.887978
min,NaN,NaN,1.000000e+03,NaN,NaN,4.000000,NaN,1953.000000,4700.000000
25%,NaN,NaN,7.300000e+04,NaN,NaN,5.000000,NaN,2012.000000,36000.000000
50%,NaN,NaN,1.230000e+05,NaN,NaN,7.000000,NaN,2017.000000,55000.000000
75%,NaN,NaN,1.850000e+05,NaN,NaN,9.000000,NaN,2019.000000,89000.000000


Najem/madhabiya naamel scaling wahdi bideya yrodhom lkol presque fard plage entre (1-100 mathalan) w nbadel asemi les colonnes accordingly fel prix wel kilometrage 

In [44]:
clean_data=data
clean_data.to_csv("Initial_Clean_Data.csv", index=False)

# distinguishing categorical data  

In [45]:
# Encode categorical variables

categorical_cols = ['Marque', 'Modèle', 'Carrosserie', 'Carburant', 'Boite Vitesse']
encoded_value_mappings = {}

for col in categorical_cols:
    label_encoder = LabelEncoder()
    clean_data.loc[:, col] = label_encoder.fit_transform(clean_data[col])

    encoded_value_mappings[col] = {label: category for label, category in enumerate(label_encoder.classes_)}

def save_mappings_to_csv(encoded_value_mappings, file_path):
    data = []
    for col, mapping in encoded_value_mappings.items():
        for encoded_value, categorical_label in mapping.items():
            data.append([col, encoded_value, categorical_label])

    df = pd.DataFrame(data, columns=['Column', 'Encoded_Value', 'Categorical_Label'])
    df.to_csv(file_path, index=False)


In [46]:
file_path = "encoded_value_mappings.csv"

# Call the function to save the mappings to a CSV file
save_mappings_to_csv(encoded_value_mappings, file_path)

In [47]:
mapped= pd.read_csv("/kaggle/working/encoded_value_mappings.csv")
mapped


,Column,Encoded_Value,Categorical_Label
0,Marque,0,AC
1,Marque,1,Alfa Romeo
2,Marque,2,Audi
3,Marque,3,Autres
4,Marque,4,BAIC YX
...,...,...,...
630,Carburant,5,Hybride essence
631,Carburant,6,Hybride rechargeable diesel
632,Carburant,7,Hybride rechargeable essence
633,Boite Vitesse,0,Automatique


In [48]:
# Shuffle the dataset
data_shuffled = shuffle(clean_data, random_state=13)
# Save the dataframe to a CSV file in the notebook's working directory
data_shuffled.to_csv("Final_Shuffled_Clean_Data.csv", index=False)

data_shuffled.head()

,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix
8078,47,160,170000,2,0,19,2,2008,78000
8696,47,143,119000,2,0,8,2,2015,115000
7228,47,241,83000,7,0,9,9,2021,193000
13052,20,414,91000,2,1,6,2,2019,44000
8001,47,151,138000,2,0,10,2,2016,145000


In [49]:
display(data_shuffled)

,Marque,Modèle,Kilométrage,Carburant,Boite Vitesse,Puissance Fiscale,Carrosserie,Année,Prix
8078,47,160,170000,2,0,19,2,2008,78000
8696,47,143,119000,2,0,8,2,2015,115000
7228,47,241,83000,7,0,9,9,2021,193000
13052,20,414,91000,2,1,6,2,2019,44000
8001,47,151,138000,2,0,10,2,2016,145000
...,...,...,...,...,...,...,...,...,...
4230,24,232,120000,2,1,5,10,2019,25000
13783,23,404,80000,0,0,7,8,2018,120000
120,47,143,183000,0,1,7,2,2009,47498
9035,5,458,150000,2,0,7,2,2014,70000


In [50]:
# import seaborn as sns 

In [51]:
# sns.pairplot(data_shuffled[["Marque", "Modèle", "Kilométrage", "Carburant","Boite Vitesse","Puissance Fiscale","Carrosserie","Année","Prix"]], diag_kind="kde")